In [2]:
import os 
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

from collections import Counter
# Lemmatize with POS Tag
from sklearn.metrics.pairwise import linear_kernel



In [3]:
f = open('Store_Description_contentBased.pckl', 'rb')
tf_idf,indices = pickle.load(f)
f.close()

In [4]:
D = np.load('tf_idf_vector_matrix.npy') # load

## Import Data

In [6]:
md = pd.read_csv('../the-movies-dataset/movies_metadata_merge_tmvdb.csv')

## Matching Score with movie tf_idf value

In [7]:
def get_movie_query(title):
    idx = indices[title]
    tokens = []
    
    for i in tf_idf:
        if i[0] == idx: tokens.append(i[1])
    
    return tokens 

In [8]:
def matching_score_ver1(title):
    tokens = get_movie_query(title)
    
    query_weights = {}
    
    for key in tf_idf:
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
   
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse = True)
    # x[1] is matching score value
    
    print("")
    
    return query_weights[1:10]

In [9]:
matching_score_ver1("The Dark Knight")

[(7931, 0.9648405846944212),
 (8227, 0.9437619594662455),
 (7760, 0.8632398290551863),
 (524, 0.7817081006616236),
 (2341, 0.778908307175292),
 (1113, 0.7550854622596759),
 (7565, 0.713537500133866),
 (6144, 0.6581826358297194),
 (2579, 0.6538831528555861)]

## Cosine similarity with Movie vector 

In [10]:
# hàm tính đố tương đồng giữa 2 vector 
def cosine_sim_calculate(a,b):
    cos_sim = np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [11]:
def cosine_similarity_ver2(k,title):
    idx = indices[title]
    d_cosines = []
    
    for d in D:
        d_cosines.append(cosine_sim_calculate(D[idx],d))
    
    out = np.array(d_cosines).argsort()[::-1]
    print("")
    
    return out[1:k]

In [12]:
cosine_similarity_ver2(11,"The Dark Knight")

array([7931, 8165,  132, 8227, 6144, 2579, 1113,  524, 7901, 7565])